In [1]:
import configparser
from datetime import datetime
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col

In [2]:
spark = SparkSession \
    .builder \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
    .getOrCreate()

22/03/06 12:43:14 WARN Utils: Your hostname, MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.0.29 instead (on interface en0)
22/03/06 12:43:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/Users/eescobar/opt/anaconda3/envs/project_data_modeling_postgres/lib/python3.8/site-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/eescobar/.ivy2/cache
The jars for the packages stored in: /Users/eescobar/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-d48f107f-066e-4d1e-8936-3663db56b916;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;2.7.0 in central
	found org.apache.hadoop#hadoop-common;2.7.0 in central
	found org.apache.hadoop#hadoop-annotations;2.7.0 in central
	found com.google.guava#guava;11.0.2 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found commons-cli#commons-cli;1.2 in central
	found org.apache.commons#commons-math3;3.1.1 in central
	found xmlenc#xmlenc;0.52 in central
	found commons-httpclient#commons-httpclient;3.1 in central
	found commons-logging#commons-logging;1.1.3 in central
	found commons-codec#commons-codec;1.4 in central
	found commons-io#commons-io;2.4 in central
	found commons-net#commons-net;3.1 in central
	found commons-collections#commons-c

In [70]:
song_data = "data/song_data/*/*/*/*.json"

In [71]:
df = spark.read.json(song_data)

In [72]:
df.printSchema()

root
 |-- artist_id: string (nullable = true)
 |-- artist_latitude: double (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_longitude: double (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- num_songs: long (nullable = true)
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: long (nullable = true)



In [6]:
df.select("artist_id", "artist_name", "artist_location", "artist_latitude", "artist_longitude").show(5)

+------------------+--------------------+-----------------+---------------+----------------+
|         artist_id|         artist_name|  artist_location|artist_latitude|artist_longitude|
+------------------+--------------------+-----------------+---------------+----------------+
|ARDR4AC1187FB371A1|Montserrat Caball...|                 |           null|            null|
|AREBBGV1187FB523D2|Mike Jones (Featu...|      Houston, TX|           null|            null|
|ARMAC4T1187FB3FA4C|The Dillinger Esc...|Morris Plains, NJ|       40.82624|       -74.47995|
|ARPBNLO1187FB3D52F|            Tiny Tim|     New York, NY|       40.71455|       -74.00712|
|ARDNS031187B9924F0|          Tim Wilson|          Georgia|       32.67828|       -83.22295|
+------------------+--------------------+-----------------+---------------+----------------+
only showing top 5 rows



In [7]:
df.select("song_id", "title", "artist_id", "year", "duration").show(5)

+------------------+--------------------+------------------+----+---------+
|           song_id|               title|         artist_id|year| duration|
+------------------+--------------------+------------------+----+---------+
|SOBAYLL12A8C138AF9|Sono andati? Fing...|ARDR4AC1187FB371A1|   0|511.16363|
|SOOLYAZ12A6701F4A6|Laws Patrolling (...|AREBBGV1187FB523D2|   0|173.66159|
|SOBBUGU12A8C13E95D|Setting Fire to S...|ARMAC4T1187FB3FA4C|2004|207.77751|
|SOAOIBZ12AB01815BE|I Hold Your Hand ...|ARPBNLO1187FB3D52F|2000| 43.36281|
|SONYPOM12A8C13B2D7|I Think My Wife I...|ARDNS031187B9924F0|2005|186.48771|
+------------------+--------------------+------------------+----+---------+
only showing top 5 rows



In [73]:
songs_table = df.select("song_id", "title", "artist_id", "year", "duration").dropDuplicates()

In [17]:
songs_table = songs_table.write.partitionBy("year", "artist_id").parquet("./data/songs_table.parquet")

22/03/06 12:56:56 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers
22/03/06 12:56:56 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 84.47% for 8 writers
22/03/06 12:56:56 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers
22/03/06 12:56:57 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers
22/03/06 12:56:57 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 84.47% for 8 writers
22/03/06 12:56:57 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers
22/03/06 12:56:58 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014

In [18]:
parDF = spark.read.parquet("./data/songs_table.parquet")

In [19]:
parDF.printSchema()

root
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- year: integer (nullable = true)
 |-- artist_id: string (nullable = true)



In [21]:
parDF.createOrReplaceTempView("songs_table")

In [22]:
spark.sql("SELECT * FROM songs_table").show(5)

+------------------+--------------------+---------+----+------------------+
|           song_id|               title| duration|year|         artist_id|
+------------------+--------------------+---------+----+------------------+
|SOAOIBZ12AB01815BE|I Hold Your Hand ...| 43.36281|2000|ARPBNLO1187FB3D52F|
|SONYPOM12A8C13B2D7|I Think My Wife I...|186.48771|2005|ARDNS031187B9924F0|
|SODREIN12A58A7F2E5|A Whiter Shade Of...|326.00771|   0|ARLTWXK1187FB5A3F8|
|SOYMRWW12A6D4FAB14|The Moon And I (O...| 267.7024|   0|ARKFYS91187B98E58F|
|SOWQTQZ12A58A7B63E|Streets On Fire (...|279.97995|   0|ARPFHN61187FB575F6|
+------------------+--------------------+---------+----+------------------+
only showing top 5 rows



In [25]:
df_logs = spark.read.json("./data/log-data/*.json")

In [26]:
df_logs.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [28]:
df_logs.filter(df_logs.page == "NextSong").limit(5).toPandas()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Harmonia,Logged In,Ryan,M,0,Smith,655.77751,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,Sehr kosmisch,200,1542241826796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26
1,The Prodigy,Logged In,Ryan,M,1,Smith,260.07465,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,The Big Gundown,200,1542242481796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26
2,Train,Logged In,Ryan,M,2,Smith,205.45261,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,Marry Me,200,1542242741796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26
3,Sony Wonder,Logged In,Samuel,M,0,Gonzalez,218.06975,free,"Houston-The Woodlands-Sugar Land, TX",PUT,NextSong,1.540493e+12,597,Blackbird,200,1542253449796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",61
4,Van Halen,Logged In,Tegan,F,2,Levine,289.38404,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,602,Best Of Both Worlds (Remastered Album Version),200,1542260935796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80


In [30]:
df_logs.select("userId", "firstName", "lastName", "gender", "level").withColumnRenamed("userId","user_id").withColumnRenamed().limit(10).toPandas()

,user_id,firstName,lastName,gender,level
0,26,Ryan,Smith,M,free
1,26,Ryan,Smith,M,free
2,26,Ryan,Smith,M,free
3,9,Wyatt,Scott,M,free
4,12,Austin,Rosales,M,free
5,61,Samuel,Gonzalez,M,free
6,61,Samuel,Gonzalez,M,free
7,,None,None,None,paid
8,80,Tegan,Levine,F,paid
9,80,Tegan,Levine,F,paid


In [51]:
from pyspark.sql.types import TimestampType

In [52]:
@udf(returnType= TimestampType())
def get_timestamp(ts):
    return datetime.fromtimestamp(ts/1000.0)

In [53]:
df_logs.select(get_timestamp("ts").alias("start_time")).show(5)

+--------------------+
|          start_time|
+--------------------+
|2018-11-15 01:30:...|
|2018-11-15 01:41:...|
|2018-11-15 01:45:...|
|2018-11-15 02:57:...|
|2018-11-15 04:29:...|
+--------------------+
only showing top 5 rows



In [54]:
new_df = df_logs.withColumn("start_time", get_timestamp(df_logs.ts))

In [55]:
new_df.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)
 |-- start_time: timestamp (nullable = true)



In [56]:
import pyspark.sql.functions as F

In [57]:
new_df = new_df.withColumn("hour", F.hour(new_df.start_time))

In [58]:
new_df.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)
 |-- start_time: timestamp (nullable = true)
 |-- hour: integer (nullable = true)



In [60]:
new_df.toPandas()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId,start_time,hour
0,Harmonia,Logged In,Ryan,M,0,Smith,655.77751,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,Sehr kosmisch,200,1542241826796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26,2018-11-15 01:30:26.796,1
1,The Prodigy,Logged In,Ryan,M,1,Smith,260.07465,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,The Big Gundown,200,1542242481796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26,2018-11-15 01:41:21.796,1
2,Train,Logged In,Ryan,M,2,Smith,205.45261,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,Marry Me,200,1542242741796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26,2018-11-15 01:45:41.796,1
3,None,Logged In,Wyatt,M,0,Scott,NaN,free,"Eureka-Arcata-Fortuna, CA",GET,Home,1.540872e+12,563,None,200,1542247071796,Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7....,9,2018-11-15 02:57:51.796,2
4,None,Logged In,Austin,M,0,Rosales,NaN,free,"New York-Newark-Jersey City, NY-NJ-PA",GET,Home,1.541060e+12,521,None,200,1542252577796,Mozilla/5.0 (Windows NT 6.1; rv:31.0) Gecko/20...,12,2018-11-15 04:29:37.796,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8051,Black Eyed Peas,Logged In,Sylvie,F,0,Cruz,214.93506,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",PUT,NextSong,1.540266e+12,9,Pump It,200,1541108520796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",10,2018-11-01 22:42:00.796,22
8052,None,Logged In,Ryan,M,0,Smith,NaN,free,"San Jose-Sunnyvale-Santa Clara, CA",GET,Home,1.541017e+12,169,None,200,1541109015796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26,2018-11-01 22:50:15.796,22
8053,Fall Out Boy,Logged In,Ryan,M,1,Smith,200.72444,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,169,Nobody Puts Baby In The Corner,200,1541109125796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26,2018-11-01 22:52:05.796,22
8054,M.I.A.,Logged In,Ryan,M,2,Smith,233.71710,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,169,Mango Pickle Down River (With The Wilcannia Mob),200,1541109325796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26,2018-11-01 22:55:25.796,22


In [66]:
new_df = new_df.withColumn("hour", F.hour(new_df.start_time))
new_df = new_df.withColumn("day", F.dayofmonth(new_df.start_time))
new_df = new_df.withColumn("week", F.weekofyear(new_df.start_time))
new_df = new_df.withColumn("month", F.month(new_df.start_time))
new_df = new_df.withColumn("year", F.year(new_df.start_time))
new_df = new_df.withColumn("weekday", F.date_format(new_df.start_time, "E"))

In [67]:
new_df.toPandas()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,...,ts,userAgent,userId,start_time,hour,day,week,month,year,weekday
0,Harmonia,Logged In,Ryan,M,0,Smith,655.77751,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,...,1542241826796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26,2018-11-15 01:30:26.796,1,15,46,11,2018,Thu
1,The Prodigy,Logged In,Ryan,M,1,Smith,260.07465,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,...,1542242481796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26,2018-11-15 01:41:21.796,1,15,46,11,2018,Thu
2,Train,Logged In,Ryan,M,2,Smith,205.45261,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,...,1542242741796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26,2018-11-15 01:45:41.796,1,15,46,11,2018,Thu
3,None,Logged In,Wyatt,M,0,Scott,NaN,free,"Eureka-Arcata-Fortuna, CA",GET,...,1542247071796,Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7....,9,2018-11-15 02:57:51.796,2,15,46,11,2018,Thu
4,None,Logged In,Austin,M,0,Rosales,NaN,free,"New York-Newark-Jersey City, NY-NJ-PA",GET,...,1542252577796,Mozilla/5.0 (Windows NT 6.1; rv:31.0) Gecko/20...,12,2018-11-15 04:29:37.796,4,15,46,11,2018,Thu
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8051,Black Eyed Peas,Logged In,Sylvie,F,0,Cruz,214.93506,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",PUT,...,1541108520796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",10,2018-11-01 22:42:00.796,22,1,44,11,2018,Thu
8052,None,Logged In,Ryan,M,0,Smith,NaN,free,"San Jose-Sunnyvale-Santa Clara, CA",GET,...,1541109015796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26,2018-11-01 22:50:15.796,22,1,44,11,2018,Thu
8053,Fall Out Boy,Logged In,Ryan,M,1,Smith,200.72444,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,...,1541109125796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26,2018-11-01 22:52:05.796,22,1,44,11,2018,Thu
8054,M.I.A.,Logged In,Ryan,M,2,Smith,233.71710,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,...,1541109325796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26,2018-11-01 22:55:25.796,22,1,44,11,2018,Thu


In [74]:
from pyspark.sql.functions import monotonically_increasing_id

songplays_table = new_df.join(songs_table, (new_df.song == songs_table.title) & (new_df.artist == songs_table.artist_name) & (new_df.length == songs_table.duration), 'left_outer')\
    .select(new_df.start_time, new_df.userId, new_df.level, songs_table.song_id, songs_table.artist_id, new_df.sessionId, new_df.location, new_df.userAgent, new_df.year, new_df.month)\
    .withColumn("songplay_id", monotonically_increasing_id())

AttributeError: 'DataFrame' object has no attribute 'artist_name'

In [75]:
songs_table.show(5)

+------------------+--------------------+------------------+----+---------+
|           song_id|               title|         artist_id|year| duration|
+------------------+--------------------+------------------+----+---------+
|SOGOSOV12AF72A285E|   ¿Dónde va Chichi?|ARGUVEV1187B98BA17|1997|313.12934|
|SOTTDKS12AB018D69B|It Wont Be Christmas|ARMBR4Y1187B9990EB|   0|241.47546|
|SOBBUGU12A8C13E95D|Setting Fire to S...|ARMAC4T1187FB3FA4C|2004|207.77751|
|SOIAZJW12AB01853F1|          Pink World|AR8ZCNI1187B9A069B|1984|269.81832|
|SONYPOM12A8C13B2D7|I Think My Wife I...|ARDNS031187B9924F0|2005|186.48771|
+------------------+--------------------+------------------+----+---------+
only showing top 5 rows



In [76]:
song_df = spark.read.json("data/song_data/*/*/*/*.json")

In [77]:
song_df.printSchema()

root
 |-- artist_id: string (nullable = true)
 |-- artist_latitude: double (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_longitude: double (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- num_songs: long (nullable = true)
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: long (nullable = true)



In [89]:
songplays_table = new_df.join(song_df, (new_df.song == song_df.title) & (new_df.artist == song_df.artist_name) & (new_df.length == song_df.duration), 'left_outer')\
    .select(new_df.start_time, new_df.userId, new_df.level, song_df.song_id, song_df.title, song_df.artist_id, new_df.sessionId, new_df.location, new_df.userAgent, new_df.year, new_df.month, new_df.day)\
    .withColumn("songplay_id", monotonically_increasing_id())

In [90]:
songplays_table.filter(songplays_table.song_id.isNotNull()).toPandas()

,start_time,userId,level,song_id,title,artist_id,sessionId,location,userAgent,year,month,day,songplay_id
0,2018-11-21 22:56:47.796,15,paid,SOZCTXZ12AB0182364,Setanta matins,AR5KOSW1187FB35FF4,818,"Chicago-Naperville-Elgin, IL-IN-WI","""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",2018,11,21,1016


In [82]:
songplays_table.toPandas()

,start_time,userId,level,song_id,artist_id,sessionId,location,userAgent,year,month,songplay_id
0,2018-11-15 01:30:26.796,26,free,None,None,583,"San Jose-Sunnyvale-Santa Clara, CA","""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",2018,11,0
1,2018-11-15 01:41:21.796,26,free,None,None,583,"San Jose-Sunnyvale-Santa Clara, CA","""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",2018,11,1
2,2018-11-15 01:45:41.796,26,free,None,None,583,"San Jose-Sunnyvale-Santa Clara, CA","""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",2018,11,2
3,2018-11-15 02:57:51.796,9,free,None,None,563,"Eureka-Arcata-Fortuna, CA",Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7....,2018,11,3
4,2018-11-15 04:29:37.796,12,free,None,None,521,"New York-Newark-Jersey City, NY-NJ-PA",Mozilla/5.0 (Windows NT 6.1; rv:31.0) Gecko/20...,2018,11,4
...,...,...,...,...,...,...,...,...,...,...,...
8051,2018-11-01 22:42:00.796,10,free,None,None,9,"Washington-Arlington-Alexandria, DC-VA-MD-WV","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",2018,11,60129542213
8052,2018-11-01 22:50:15.796,26,free,None,None,169,"San Jose-Sunnyvale-Santa Clara, CA","""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",2018,11,60129542214
8053,2018-11-01 22:52:05.796,26,free,None,None,169,"San Jose-Sunnyvale-Santa Clara, CA","""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",2018,11,60129542215
8054,2018-11-01 22:55:25.796,26,free,None,None,169,"San Jose-Sunnyvale-Santa Clara, CA","""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",2018,11,60129542216


In [91]:
song_data = "data/song_data/*/*/*/*.json"

In [92]:
df = spark.read.json(song_data)

In [93]:
songs_table = df.select("song_id", "title", "artist_id", "year", "duration").dropDuplicates()

In [94]:
artists_table = df.select("artist_id",
                          col("artist_name").alias("name"),
                          col("artist_location").alias("location"),
                          col("artist_latitude").alias("latitude"),
                          col("artist_longitude").alias("longitude")).dropDuplicates()

In [96]:
log_data = "data/log-data/*.json"

In [98]:
df = spark.read.json(log_data)

In [99]:
users_table = df.select(col("userId").alias("user_id"),
                        col("firstName").alias("first_name"),
                        col("lastName").alias("last_name"), "gender", "level")

In [100]:
@udf(returnType=TimestampType())
def get_timestamp(ts):
    return datetime.fromtimestamp(ts/1000.0)

df = df.withColumn("start_time", get_timestamp(df.ts))
df = df.withColumn("hour", F.hour(df.start_time))
df = df.withColumn("day", F.dayofmonth(df.start_time))
df = df.withColumn("week", F.weekofyear(df.start_time))
df = df.withColumn("month", F.month(df.start_time))
df = df.withColumn("year", F.year(df.start_time))
df = df.withColumn("weekday", F.date_format(df.start_time, "E"))

In [105]:
time_table = df.select("start_time", "hour", "day", "week", "month", "year", "weekday")\
    .dropDuplicates()

In [114]:
song_artist_table = songs_table.join(artists_table, "artist_id", 'left_outer')

In [115]:
song_artist_table.toPandas()

,artist_id,song_id,title,year,duration,name,location,latitude,longitude
0,ARGUVEV1187B98BA17,SOGOSOV12AF72A285E,¿Dónde va Chichi?,1997,313.12934,Sierra Maestra,,NaN,NaN
1,ARMBR4Y1187B9990EB,SOTTDKS12AB018D69B,It Wont Be Christmas,0,241.47546,David Martin,California - SF,37.77916,-122.42005
2,ARMAC4T1187FB3FA4C,SOBBUGU12A8C13E95D,Setting Fire to Sleeping Giants,2004,207.77751,The Dillinger Escape Plan,"Morris Plains, NJ",40.82624,-74.47995
3,AR8ZCNI1187B9A069B,SOIAZJW12AB01853F1,Pink World,1984,269.81832,Planet P Project,,NaN,NaN
4,ARDNS031187B9924F0,SONYPOM12A8C13B2D7,I Think My Wife Is Running Around On Me (Taco ...,2005,186.48771,Tim Wilson,Georgia,32.67828,-83.22295
...,...,...,...,...,...,...,...,...,...
66,ARAJPHH1187FB5566A,SOYTPEP12AB0180E7B,Twist and Shout,1964,164.80608,The Shangri-Las,"Queens, NY",40.70380,-73.83168
67,AR62SOJ1187FB47BB5,SOGVQGJ12AB017F169,Ten Tonne,2005,337.68444,Chase & Status,,NaN,NaN
68,AREDL271187FB40F44,SOPEGZN12AB0181B3D,Get Your Head Stuck On Your Neck,0,45.66159,Soul Mekanik,,NaN,NaN
69,AR5KOSW1187FB35FF4,SOZCTXZ12AB0182364,Setanta matins,0,269.58322,Elena,Dubai UAE,49.80388,15.47491


In [110]:
df.toPandas()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,...,ts,userAgent,userId,start_time,hour,day,week,month,year,weekday
0,Harmonia,Logged In,Ryan,M,0,Smith,655.77751,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,...,1542241826796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26,2018-11-15 01:30:26.796,1,15,46,11,2018,Thu
1,The Prodigy,Logged In,Ryan,M,1,Smith,260.07465,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,...,1542242481796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26,2018-11-15 01:41:21.796,1,15,46,11,2018,Thu
2,Train,Logged In,Ryan,M,2,Smith,205.45261,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,...,1542242741796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26,2018-11-15 01:45:41.796,1,15,46,11,2018,Thu
3,None,Logged In,Wyatt,M,0,Scott,NaN,free,"Eureka-Arcata-Fortuna, CA",GET,...,1542247071796,Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7....,9,2018-11-15 02:57:51.796,2,15,46,11,2018,Thu
4,None,Logged In,Austin,M,0,Rosales,NaN,free,"New York-Newark-Jersey City, NY-NJ-PA",GET,...,1542252577796,Mozilla/5.0 (Windows NT 6.1; rv:31.0) Gecko/20...,12,2018-11-15 04:29:37.796,4,15,46,11,2018,Thu
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8051,Black Eyed Peas,Logged In,Sylvie,F,0,Cruz,214.93506,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",PUT,...,1541108520796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",10,2018-11-01 22:42:00.796,22,1,44,11,2018,Thu
8052,None,Logged In,Ryan,M,0,Smith,NaN,free,"San Jose-Sunnyvale-Santa Clara, CA",GET,...,1541109015796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26,2018-11-01 22:50:15.796,22,1,44,11,2018,Thu
8053,Fall Out Boy,Logged In,Ryan,M,1,Smith,200.72444,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,...,1541109125796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26,2018-11-01 22:52:05.796,22,1,44,11,2018,Thu
8054,M.I.A.,Logged In,Ryan,M,2,Smith,233.71710,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,...,1541109325796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26,2018-11-01 22:55:25.796,22,1,44,11,2018,Thu


In [116]:
songplays_table = df.join(song_artist_table, (df.song == song_artist_table.title) & (df.artist == song_artist_table.name) & (df.length == song_artist_table.duration), 'left_outer')\
    .select(df.start_time, col(df.userId).alias("user_id"), df.level, song_artist_table.song_id, song_artist_table.artist_id, col(df.sessionId).alias("session_id"), df.location, col(df.userAgent).alias("user_agent"))\
    .withColumn("songplay_id", monotonically_increasing_id())

In [118]:
songplays_table.toPandas()

,start_time,userId,level,song_id,artist_id,sessionId,location,userAgent,year,month,day,songplay_id
0,2018-11-20 15:21:35.796,44,paid,None,None,639,"Waterloo-Cedar Falls, IA",Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,2018,11,20,0
1,2018-11-22 00:36:31.796,15,paid,None,None,818,"Chicago-Naperville-Elgin, IL-IN-WI","""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",2018,11,22,1
2,2018-11-21 13:29:49.796,15,paid,None,None,764,"Chicago-Naperville-Elgin, IL-IN-WI","""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",2018,11,21,2
3,2018-11-05 14:42:46.796,44,paid,None,None,269,"Waterloo-Cedar Falls, IA",Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,2018,11,5,3
4,2018-11-04 08:00:08.796,25,paid,None,None,128,"Marinette, WI-MI","""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",2018,11,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...
8051,2018-11-16 19:17:31.796,97,paid,None,None,633,"Lansing-East Lansing, MI","""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",2018,11,16,1709396983829
8052,2018-11-05 18:38:16.796,73,paid,None,None,255,"Tampa-St. Petersburg-Clearwater, FL","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",2018,11,5,1709396983830
8053,2018-11-23 21:15:47.796,85,paid,None,None,891,"Red Bluff, CA","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_...",2018,11,23,1709396983831
8054,2018-11-02 21:43:17.796,3,free,None,None,112,"Saginaw, MI",Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,2018,11,2,1709396983832


In [119]:
songplays_table.filter(songplays_table.song_id.isNotNull()).toPandas()

,start_time,userId,level,song_id,artist_id,sessionId,location,userAgent,year,month,day,songplay_id
0,2018-11-21 22:56:47.796,15,paid,SOZCTXZ12AB0182364,AR5KOSW1187FB35FF4,818,"Chicago-Naperville-Elgin, IL-IN-WI","""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",2018,11,21,1589137899536
